# Day 19

In [1]:
from ipynb.fs.defs.utils import read_lines 
import re

In [2]:
puzzle_input = read_lines('day19.txt')

In [3]:
test_input = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}""".splitlines()

In [4]:
rule_pattern = re.compile(r'(([xmas])(<|>)(\d+):([a-zAR]+))|([a-zAR]+)')
def parse_rule(inp):
    m = rule_pattern.match(inp)
    if m.group(6):
        return m.group(6)
    else:
        var, comp, val, action = m.group(2,3,4,5)
        return var, comp, int(val), action

In [5]:
test_rule =parse_rule('s>2770:qs')
test_rule

('s', '>', 2770, 'qs')

In [6]:
parse_rule('R')

'R'

In [7]:
wf_pattern = re.compile(r'([a-z]+)\{(.+)\}')
def parse_workflow(inp):
    rules = []
    m = wf_pattern.match(inp)
    name, rules = m.group(1, 2)
    rules = [parse_rule(r) for r in rules.split(',')]
    return name, rules[:-1], rules[-1]

In [8]:
test_wf = parse_workflow('qqz{s>2770:qs,m<1801:hdj,R}')
test_wf

('qqz', [('s', '>', 2770, 'qs'), ('m', '<', 1801, 'hdj')], 'R')

In [9]:
def parse_kvp(inp):
    k, v = inp.split('=')
    return k, int(v)

In [10]:
def parse_part(inp):
    part = {}
    for kvp in inp[1:-1].split(','):
        k,v = parse_kvp(kvp)
        part[k] = v
    return part

In [11]:
test_part = parse_part('{x=787,m=2655,a=1222,s=2876}')
test_part

{'x': 787, 'm': 2655, 'a': 1222, 's': 2876}

In [12]:
def parse_input(inp):
    wfs = {}
    parts = []
    empty_encountered = False
    for i in range(len(inp)):
        if len(inp[i]) == 0:
            empty_encountered = True
        elif empty_encountered:
            parts.append(parse_part(inp[i]))
        else:
            name, rules, default = parse_workflow(inp[i])
            wfs[name] = (rules, default)
    return wfs, parts

In [13]:
def matches_rule(part, rule):
    var, comp, val, _ = rule
    if comp == '>':
        return part[var] > val
    if comp == '<':
        return part[var] < val
    print(f'Unknown comparator in {rule}')

In [14]:
def apply_wf(part, wf):
    rules, default = wf
    for r in rules:
        if matches_rule(part, r):
            return r[3]
    return default

In [15]:
apply_wf(test_part, test_wf[1:])

'qs'

In [16]:
def process_part(part, workflows):
    wf = 'in'
    while wf != 'R' and wf != 'A':
        wf = apply_wf(part, workflows[wf])
    return wf

In [17]:
def rating_number(part):
    return sum(part.values())

In [18]:
def part1(inp):
    wfs, parts = parse_input(inp)
    return sum([rating_number(p) for p in parts if process_part(p, wfs) == 'A'])

In [19]:
part1(test_input)

19114

In [20]:
part1(puzzle_input)

342650

In [21]:
def apply_rule(p_range, rule):
    attr, comp, v_rule, action = rule
    v_from, v_to = p_range[attr]
    if v_from >= v_rule and comp == '<' or v_to - 1 <= v_rule and comp == '>':
        # Rule doesn't apply to range
        return None, p_range
    if v_from > v_rule and comp == '>' or v_to - 1 < v_rule and comp == '<':
        # Whole rules applies to range
        return (p_range, action), None
    else:
        if comp == '<':
            match_range = (v_from, v_rule)
            leftover_range = (v_rule, v_to)
        if comp == '>':
            match_range = (v_rule + 1, v_to)
            leftover_range = (v_from, v_rule + 1)
        match_part = p_range.copy()
        leftover_part = p_range.copy()
        match_part[attr] = match_range
        leftover_part[attr] = leftover_range
        return (match_part, action), leftover_part

In [22]:
test_rule

('s', '>', 2770, 'qs')

In [23]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2771)}, ('s', '>', 2770, 'qs'))

(None, {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2771)})

In [24]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2772)}, ('s', '>', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2771, 2772)},
  'qs'),
 {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2771)})

In [25]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 3000)}, ('s', '>', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2771, 3000)},
  'qs'),
 {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 2771)})

In [26]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2771, 3000)}, ('s', '>', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2771, 3000)},
  'qs'),
 None)

In [27]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2770)}, ('s', '<', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2770)}, 'qs'),
 None)

In [28]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2771)}, ('s', '<', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (0, 2770)}, 'qs'),
 {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 2771)})

In [29]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2769, 3000)}, ('s', '<', 2770, 'qs'))

(({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2769, 2770)},
  'qs'),
 {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 3000)})

In [30]:
apply_rule({'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 3000)}, ('s', '<', 2770, 'qs'))

(None, {'x': (0, 787), 'm': (2655, 3500), 'a': (500, 1222), 's': (2770, 3000)})

In [31]:
def apply_wf_range(p_range, wf):
    rules, default = wf
    out_ranges = []
    leftover = p_range
    for r in rules:
        if leftover:
            match, leftover = apply_rule(leftover, r)
            if match:
                out_ranges.append(match)
        else:
            break
    if leftover:
        out_ranges.append((leftover, default))
    return out_ranges

In [32]:
apply_wf_range({'x': (0, 787), 'm': (1000, 3500), 'a': (500, 1222), 's': (2770, 3000)}, ([('s', '>', 2770, 'qs'), ('m', '<', 1801, 'hdj')], 'R'))

[({'x': (0, 787), 'm': (1000, 3500), 'a': (500, 1222), 's': (2771, 3000)},
  'qs'),
 ({'x': (0, 787), 'm': (1000, 1801), 'a': (500, 1222), 's': (2770, 2771)},
  'hdj'),
 ({'x': (0, 787), 'm': (1801, 3500), 'a': (500, 1222), 's': (2770, 2771)},
  'R')]

In [33]:
def range_rating_number(part):
    prod = 1
    factors = [t - f for f, t in part.values()]
    for fac in factors:
        prod *= fac
    return prod

In [34]:
range_rating_number({'x': (0, 787), 'm': (1000, 3500), 'a': (500, 1222), 's': (2770, 3000)})

326723050000

In [35]:
def part2(inp, v=False):
    wfs, parts = parse_input(inp)
    todo = [({'x': (1, 4001), 'm': (1, 4001), 'a': (1, 4001), 's': (1, 4001)}, 'in')]
    res_parts = []
    while len(todo) > 0:
        next_todo = []
        for part, wf in todo:
            if wf == 'A':
                res_parts.append(part)
            elif wf == 'R':
                continue
            else:
                next_todo += apply_wf_range(part, wfs[wf])
        todo = next_todo
    if v: print(res_parts)
    return sum(range_rating_number(part) for part in res_parts)

In [36]:
part2(test_input)

167409079868000

In [37]:
part2(puzzle_input)

130303473508222